In [19]:
import numpy as np
from docplex.mp.model import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import time
%matplotlib inline

class Box:
    def __init__(self, id, x, y, z, w, d, h, bin):
        self.id = id
        self.x = x
        self.y = y
        self.z = z
        self.w = w
        self.h = h
        self.d = d
        self.bin = bin

    def to_dict(self):
        return {
            "id": self.id,
            "x": self.x,
            "y": self.y,
            "z": self.z,
            "width": self.w,
            "depth": self.d,
            "height": self.h,
            "bin": self.bin
        }
    
    def to_packer_box(self):
        return {
            "id": str(self.w) + str(self.d) + str(self.h) + "-" + str(self.id),
            "width": self.w,
            "depth": self.d,
            "height": self.h,
            "weight": 0.0,
            "maxLoad": 999.0,
            "availableInWarehouse": True,
            "onTop": False,
            "priority": 1,
            "incompatibilities": []
        }
def generateRotatedDuplices(boxes):
    newBoxes = []
    rotatedBoxes = []
    for box in boxes:
        newBoxes.append(box)
        newBoxes.append(Box(box.id, box.x, box.y, box.z, box.d, box.w, box.h, box.bin))
    return newBoxes

def parseInstance(instanceFile):
    W = 0
    D = 0
    H = 0
    boxes = []
    problemFile = open(instanceFile, 'r')
    for line in problemFile.readlines():
        commands = line.split(" ")
        args = commands[1].split(",")
        if commands[0] == "bin":
            W = int(args[0])
            D = int(args[1])
            H = int(args[2])
        if commands[0] == "box":
            box = Box(int(args[0]), 0, 0, 0, int(args[1]), int(args[2]), int(args[3]), 0)
            boxes.append(box);
    return (W, D, H, boxes)


#Utility functions (probably will be used in discretization)
def calculateOverlap(i, b1, k, b2, xk, yk):
    if b1 != b2: 
        return 0
    ximax = boxes[i].w
    yimax = boxes[i].d
    xkmax = xk + boxes[k].w
    ykmax = yk + boxes[k].d
    dx = min(ximax, xkmax) - max(0, xk)
    dy = min(yimax, ykmax) - max(0, yk)
    #value = 0 if dx<0 or dy<0 else dx*dy
    #if value != 0:
    #    print("Overlap {},{},{},{},{},{} = {}".format(i, b1, k, b2, xk, yk, value))
    return 0 if dx<0 or dy<0 else dx*dy

In [49]:
def getSolutionBox(i, boxes, solution):
    box = boxes[i]
    used_in = [b for b in B if solution.get_value(u[i][b]) == 1]
    if len(used_in) == 0:
        return None
    bin = used_in[0]
    return Box(
        box.id,
        solution.get_value(x[i]),
        solution.get_value(y[i]),
        solution.get_value(z[i]),
        box.w,
        box.d,
        box.h,
        bin)

def cuboid_data(o, size=(1,1,1)):
    X = [[[0, 1, 0], [0, 0, 0], [1, 0, 0], [1, 1, 0]],
         [[0, 0, 0], [0, 0, 1], [1, 0, 1], [1, 0, 0]],
         [[1, 0, 1], [1, 0, 0], [1, 1, 0], [1, 1, 1]],
         [[0, 0, 1], [0, 0, 0], [0, 1, 0], [0, 1, 1]],
         [[0, 1, 0], [0, 1, 1], [1, 1, 1], [1, 1, 0]],
         [[0, 1, 1], [0, 0, 1], [1, 0, 1], [1, 1, 1]]]
    X = np.array(X).astype(float)
    for i in range(3):
        X[:,:,i] *= size[i]
    X += np.array(o)
    return X

def getCubeCollection(boxes,colors=None, **kwargs):
    if not isinstance(colors,(list,np.ndarray)): colors=["C0"]*len(boxes)
    g = []
    for box in boxes:
        g.append(cuboid_data((box["x"], box["y"], box["z"]), (box["width"], box["depth"], box["height"])))
    return Poly3DCollection(np.concatenate(g), facecolors=np.repeat(colors,6), **kwargs)

def plotBox(box, ax=None,**kwargs):
    if not isinstance(colors,(list,np.ndarray)): colors=["C0"]*len(positions)
    # Plotting a cube element at position pos
    if ax !=None:
        X, Y, Z = cuboid_data((box["x"], box["y"], box["z"]), (box["width"], box["depth"], box["height"]) )
        surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, **kwargs)
        surf.patch.set_edgecolor('black')
        ax.plot_wireframe(X, Y, Z, rstride=1, cstride=1, color="black", **kwargs)

colors = ["crimson","limegreen", "navy", "purple"]

In [50]:
## Check solution
def overlaps(xmin1, xmin2, xmax1, xmax2):
    return int(round(xmax1)) > int(round(xmin2)) and int(round(xmin1)) < int(round(xmax2))

def overlapArea(boxi, boxj):
    dx = (boxj.x - boxi.x)
    dy = (boxj.y - boxi.y)
    ximax = boxi.w
    yimax = boxi.d
    xkmax = dx + boxj.w
    ykmax = dy + boxj.d
    dx = min(ximax, xkmax) - max(0, dx)
    dy = min(yimax, ykmax) - max(0, dy)
    #value = 0 if dx<0 or dy<0 else dx*dy
    #if value != 0:
    #    print("Overlap {},{},{},{},{},{} = {}".format(i, b1, k, b2, xk, yk, value))
    return 0 if dx<0 or dy<0 else dx*dy

def checkSolution(boxes, solution):
    solutions = [getSolutionBox(i, boxes, solution) for i in I]
    nonNulls = [box for box in solutions if box != None]
    if len(nonNulls) != len(originalBoxes):
        return (False, "Some boxes weren't packed")
    for box in nonNulls:
        for other in nonNulls:
            if other != box and other.bin == box.bin:
                overlapX = overlaps(box.x, other.x, box.x + box.w, other.x + other.w)
                overlapY = overlaps(box.y, other.y, box.y + box.d, other.y + other.d)
                overlapZ = overlaps(box.z, other.z, box.z + box.h, other.z + other.h)
                if overlapX and overlapY and overlapZ:
                    return (False, "Box {} overlaps box {}".format(box.id, other.id))
    for box in nonNulls:
        supp_area = 0
        if box.z > beta_s:
            for other in nonNulls:
                if other != box and other.bin == box.bin and other.z + other.h <= box.z and box.z - (other.z + other.h) <= beta_s:
                    supp_area += overlapArea(box, other)
            if supp_area < alpha_s*box.w*box.d:
                return (False, "Box {} got area support {}".format(box.id, supp_area/(box.w*box.d)))
    return (True, "")

In [ ]:
outDir = "tests/model/solutions/"
testInstanceDir = "tests/model/instances/"

prevSolution = None
with open("tests/model/summary.csv", "w") as summaryFile:
    for instanceId in np.arange(1,25):
        instanceFile = testInstanceDir + "instance-" + str(instanceId) + ".test"
        print("Starting instance " + str(instanceId))
        mdl = Model(name="3DBPP")
        problem = parseInstance(instanceFile)
        W = problem[0]
        D = problem[1]
        H = problem[2]
        V = W*D*H
        DU = 10
        bins = 1
        originalBoxes = problem[3]
        boxes = generateRotatedDuplices(originalBoxes)
        I_O = np.arange(len(originalBoxes))
        I = np.arange(len(boxes))
        B = np.arange(bins)
        alpha_s = 0.7
        beta_s = 5

        maxDim = max(max(boxes[i].w, boxes[i].d) for i in I)
        I_R = np.arange(-int(np.floor(maxDim/DU)), int(np.floor(maxDim/DU)))

        # Problem Variables
        v = [mdl.integer_var(0, 1, "v_" + str(b)) for b in B]
        u = [[mdl.integer_var(0, 1, "u_" + str(i) + "_" + str(b)) for b in B] for i in I]
        x = [mdl.continuous_var(0, W-min(boxes[i].w, boxes[i].d), "x_" + str(i)) for i in I]
        y = [mdl.continuous_var(0, D-min(boxes[i].w, boxes[i].d), "y_" + str(i)) for i in I]
        z = [mdl.continuous_var(0, H-boxes[i].h, "z_" + str(i)) for i in I]
        xp = [[mdl.integer_var(0, 1, "xp_" + str(i) + "_" + str(k)) for k in I] for i in I]
        yp = [[mdl.integer_var(0, 1, "yp_" + str(i) + "_" + str(k)) for k in I] for i in I]
        zp = [[mdl.integer_var(0, 1, "zp_" + str(i) + "_" + str(k)) for k in I] for i in I]
        zc = [[mdl.integer_var(0, 1, "zc_" + str(i) + "_" + str(k)) for k in I] for i in I]
        zmax = [mdl.continuous_var(0, H, "zmax_" + str(b)) for b in B]
        s = [[mdl.integer_var(0, 1, "s_" + str(i) + "_" + str(k)) for k in I] for i in I]
        g = [mdl.integer_var(0, 1, "g_" + str(i)) for i in I]
        sp = [[[[[mdl.integer_var(0, 1, "sp_" + str(i) + "_" + str(k)  + "_" + str(b) + "_" + str(dx) + "_" + str(dy)) for dy in I_R] for dx in I_R] for b in B ] for k in I] for i in I]

        #Objective
        mdl.minimize(mdl.sum((H*v[b] + zmax[b]) for b in B) - mdl.sum(sp[i][j][b][dx][dy] for dx in I_R for dy in I_R for b in B for i in I for j in I if calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0))                                                                                     #1

        #Constraints
        #[mdl.add_constraint(mdl.sum(u[i][b] for b in B) == 1) for i in I]
        [mdl.add_constraint(mdl.sum(u[i*2][b] for b in B) + mdl.sum(u[(i*2)+1][b] for b in B) == 1) for i in I_O]
        [mdl.add_constraint(u[i][b] <= v[b]) for b in B for i in I]

        [mdl.add_constraint(x[i] + boxes[i].w <= W) for i in I]
        [mdl.add_constraint(y[i] + boxes[i].d <= D) for i in I]
        [mdl.add_constraint(z[i] + boxes[i].h <= H) for i in I]

        [mdl.add_constraint(zmax[b] >= (z[i] + boxes[i].h) - H*(1-u[i][b])) for i in I for b in B]                                                             #9

        [mdl.add_constraint((x[i] + boxes[i].w) - x[j] <= W*(1-xp[i][j])) for i in I for j in I]
        [mdl.add_constraint(x[j] - (x[i] + boxes[i].w) + 1 <= W*xp[i][j]) for i in I for j in I]

        [mdl.add_constraint((y[i] + boxes[i].d) - y[j] <= D*(1-yp[i][j])) for i in I for j in I]
        [mdl.add_constraint(y[j] - (y[i] + boxes[i].d) + 1 <= D*yp[i][j]) for i in I for j in I]

        [mdl.add_constraint((z[i] + boxes[i].h) - z[j] <= H*(1-zp[i][j])) for i in I for j in I]
        [mdl.add_constraint(z[j] - (z[i] + boxes[i].h) + 1 <= H*zp[i][j]) for i in I for j in I]

        [mdl.add_constraint(v[b] >= v[c]) for b in B for c in B if c > b]

        [mdl.add_constraint(xp[i][j] + xp[j][i] +
                           yp[i][j] + yp[j][i] +
                           zp[i][j] + zp[j][i] >= u[i][b] + u[j][b] - 1) for i in I for j in I for b in B if not i == j]

        # Support constraint

        [mdl.add_constraint(z[j] - (z[i] + boxes[i].h) <= beta_s + H*(1-zc[i][j])) for i in I for j in I if i != j]
        [mdl.add_constraint(z[j] - (z[i] + boxes[i].h) >= -beta_s - H*(1-zc[i][j])) for i in I for j in I if i != j]
        [mdl.add_constraint(s[i][j] <= zp[i][j]) for i in I for j in I]
        [mdl.add_constraint(s[i][j] <= zc[i][j]) for i in I for j in I]
        [mdl.add_constraint(s[i][j] >= zp[i][j] + zc[i][j] - 2) for i in I for j in I]
        [mdl.add_constraint(mdl.sum(s[i][j] for j in I) <= mdl.sum(u[i][b] for b in B)) for i in I]
        [mdl.add_constraint(z[i] <= beta_s + H*(1 - g[i])) for i in I]

        [mdl.add_constraint(mdl.sum(sp[i][j][b][dx][dy] for dx in I_R for dy in I_R for b in B if calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0) <= s[i][j]) for i in I for j in I]

        [mdl.add_constraint(x[i] - x[j] >= DU*dx - 2*W*(1 - sp[i][j][b][dx][dy])) for dx in I_R for dy in I_R for b in B for i in I for j in I if i != j and calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0]
        [mdl.add_constraint(x[i] - x[j] <= DU*(dx + 1) + 2*W*(1 - sp[i][j][b][dx][dy])) for dx in I_R for dy in I_R for b in B for i in I for j in I if i != j and calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0]
        [mdl.add_constraint(y[i] - y[j] >= DU*dy - 2*D*(1 - sp[i][j][b][dx][dy])) for dx in I_R for dy in I_R for b in B for i in I for j in I if i != j and calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0]
        [mdl.add_constraint(y[i] - y[j] <= DU*(dy + 1) + 2*D*(1 - sp[i][j][b][dx][dy])) for dx in I_R for dy in I_R for b in B for i in I for j in I if i != j and calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0]

        [mdl.add_constraint(mdl.sum(calculateOverlap(j, b, i, b, dx*DU, dy*DU)*sp[j][i][b][dx][dy] for dx in I_R for dy in I_R for b in B for j in I if i != j and calculateOverlap(j, b, i, b, dx*DU, dy*DU) != 0) >= alpha_s*boxes[i].w*boxes[i].d - boxes[i].w*boxes[i].d*g[i]) for i in I]

        if prevSolution != None:
            mdl.add_mip_start(prevSolution)

        #Solve
        start = time.time()
        prevSolution = mdl.solve(log_output=True)
        end = time.time()
        elapsedTime = (end - start)

        solFile = outDir + "instance-" + str(instanceId) + ".json"
        prevSolution.export(solFile, format='json')
        
        validSolution = checkSolution(boxes, prevSolution)
        solutions = [getSolutionBox(i, boxes, prevSolution) for i in I]
        nonNulls = [box for box in solutions if box != None]
        summaryFile.write("{};{};{};{};{};{}\n".format(instanceId, elapsedTime, np.max([box.z + box.h for box in nonNulls]), np.sum([prevSolution.get_value(v[b]) for b in B]), validSolution[0], validSolution[1]))
        summaryFile.flush()
        
        #Plot solution
        if prevSolution and prevSolution.is_valid_solution():
            solutions = [getSolutionBox(i, boxes, prevSolution) for i in I]
            milpBoxes = [sol.to_dict() for sol in solutions if sol != None]
            openedBins = [b for b in B if prevSolution.get_value(v[b]) == 1]
            binPerRows = min(len(openedBins), 4)
            plotRows = int(len(openedBins)/binPerRows)
            fig = plt.figure(figsize=plt.figaspect(plotRows/binPerRows))
            idx = 0
            for bin in openedBins:
                idx += 1
                ax = fig.add_subplot(plotRows, binPerRows, idx, projection='3d')
                ax.set_title("Bin " + str(bin))
                ax.set_zlim(0, max([box["z"]+box["height"] for box in milpBoxes if box["bin"] == bin]))
                ax.set_ylim(0, D)
                ax.set_xlim(0, W)
                ax.add_collection3d(getCubeCollection([box for box in milpBoxes if box["bin"] == bin], colors=colors, edgecolor="k"))
            plt.savefig(outDir + "figures/" + "instance-" + str(instanceId) + ".pdf", bbox_inches='tight')
            plt.cla()
            plt.clf()

Starting instance 1
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 942 rows and 815 columns.
MIP Presolve modified 818 coefficients.
Aggregator did 1 substitutions.
All rows and columns eliminated.
Presolve time = 0.00 sec. (1.49 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (1.60 ticks)
Parallel b&c, 16 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (1.60 ticks)
Starting instance 2
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Retaining values of one MIP start for possible repair.
Tried aggregator 2 times.
MIP Presolve eliminated 4164 rows and 4965 columns.
MIP Presolve modified 2092 coefficients.
Aggregator did 14 sub

Starting instance 5
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Retaining values of one MIP start for possible repair.
Tried aggregator 2 times.
MIP Presolve eliminated 20622 rows and 29241 columns.
MIP Presolve modified 25111 coefficients.
Aggregator did 95 substitutions.
Reduced MIP has 12910 rows, 3616 columns, and 45220 nonzeros.
Reduced MIP has 3585 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (52.30 ticks)
Probing time = 0.02 sec. (9.16 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 12910 rows, 3616 columns, and 45220 nonzeros.
Reduced MIP has 3585 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (24.78 ticks)
Probing time = 0.02 sec. (8.75 ticks)
Clique table members: 24632.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time 

   4828     0    10079.0000    92    10152.0000      Cuts: 48    64926    0.72%
   4828     0    10079.0000   151    10152.0000     Cuts: 167    65239    0.72%
   4828     2    10079.0000    45    10152.0000    10079.0000    65239    0.72%
Elapsed time = 7.64 sec. (9318.05 ticks, tree = 0.02 MB, solutions = 6)
   4968    61    10143.0000    47    10152.0000    10079.0000    66729    0.72%
   5560   340    infeasible          10152.0000    10079.0000    78701    0.72%
   5888   364    infeasible          10152.0000    10079.0000    87514    0.72%
   6711   681    10147.0000    50    10152.0000    10079.0000    98473    0.72%
   6953   738    10079.0000    36    10152.0000    10079.0000   113535    0.72%
   8171   988    10079.0000    65    10152.0000    10079.0000   133660    0.72%
   8676  1061    infeasible          10152.0000    10079.0000   149319    0.72%
   9114   885    10079.0000    56    10152.0000    10079.0000   172829    0.72%
  10250  1193    infeasible          10152.0000 

* 13806+ 3747                        10230.0000    10078.0000             1.49%
  13846  3966    infeasible          10230.0000    10078.0000   325456    1.49%
Elapsed time = 37.03 sec. (30437.53 ticks, tree = 26.33 MB, solutions = 18)
  13906  3944    10097.2055    66    10230.0000    10078.0000   335132    1.49%
  14110  3988    10154.0000    64    10230.0000    10078.0000   340874    1.49%
  14715  4048    10151.0000    60    10230.0000    10078.0000   354295    1.49%
  15387  4202    10157.0000    43    10230.0000    10078.0000   372194    1.49%
  16020  4545    10078.0000    41    10230.0000    10078.0000   400060    1.49%
  16667  4745    10078.0000    62    10230.0000    10078.0000   423983    1.49%
  16886  4887    infeasible          10230.0000    10078.0000   439979    1.49%
  17118  5096    10078.0000    55    10230.0000    10078.0000   457186    1.49%
  17602  5101    10105.1776    55    10230.0000    10078.0000   463006    1.49%
  18116  5355    infeasible          10230.0

  55818 11833    10078.0000   106    10152.0000    10078.0000  3037049    0.73%
  57322 11998    infeasible          10152.0000    10078.0000  3098014    0.73%
  58115 12210    infeasible          10152.0000    10078.0000  3216504    0.73%
Elapsed time = 231.69 sec. (177080.74 ticks, tree = 68.87 MB, solutions = 23)
  58425 12379    infeasible          10152.0000    10078.0000  3333861    0.73%
  59281 12371    10151.0000    46    10152.0000    10078.0000  3363668    0.73%
  60482 12462    10078.0000    57    10152.0000    10078.0000  3408568    0.73%
  61179 12694    infeasible          10152.0000    10078.0000  3507554    0.73%
  61432 13022        cutoff          10152.0000    10078.0000  3628519    0.73%
  61661 12877    10143.0000    71    10152.0000    10078.0000  3704441    0.73%
  62392 12864    10078.0000    56    10152.0000    10078.0000  3761950    0.73%
  63548 12866    10078.0000    50    10152.0000    10078.0000  3771601    0.73%
  64321 13150    10151.0000    63    10152

...

 196648 38487    10092.0000    64    10153.0000    10092.0000 35588843    0.60%
Elapsed time = 7654.25 sec. (4144867.05 ticks, tree = 390.68 MB, solutions = 6)
 197354 38837    10141.0000   260    10153.0000    10092.0000 35846790    0.60%
 197741 38671    infeasible          10153.0000    10092.0000 35714502    0.60%
 197994 39169    10092.0000    89    10153.0000    10092.0000 36334560    0.60%
 198090 39067    10102.0000    46    10153.0000    10092.0000 36175610    0.60%
 198228 39227    infeasible          10153.0000    10092.0000 36377285    0.60%
 198425 39230    10092.0000    71    10153.0000    10092.0000 36580611    0.60%
 199206 39221    10092.0000    75    10153.0000    10092.0000 36547814    0.60%
 200219 39252    10105.2007    75    10153.0000    10092.0000 36629920    0.60%
 201630 39457    10092.0000    56    10153.0000    10092.0000 36789302    0.60%
 202724 39275    10092.0000    67    10153.0000    10092.0000 36683486    0.60%
Elapsed time = 7934.72 sec. (4297936.30 

 281125 54946    10092.0000    59    10153.0000    10092.0000 48787956    0.60%
 281389 55602    10103.0000    47    10153.0000    10092.0000 49137739    0.60%
 281593 55908    10092.0000    68    10153.0000    10092.0000 49356358    0.60%
 282048 55992    10092.0000    95    10153.0000    10092.0000 49525433    0.60%
 283153 56091    10092.0000    80    10153.0000    10092.0000 49523176    0.60%
 284772 55922    10092.0000    89    10153.0000    10092.0000 49609840    0.60%
 286197 56076    infeasible          10153.0000    10092.0000 49658964    0.60%
 287692 56716    10115.0000    46    10153.0000    10092.0000 50093372    0.60%
Elapsed time = 10517.84 sec. (5674742.13 ticks, tree = 552.25 MB, solutions = 6)
 289582 56976    10092.0000   112    10153.0000    10092.0000 50251048    0.60%
 290997 56777    10099.6472    52    10153.0000    10092.0000 50122501    0.60%
 292400 58598    10092.0000    65    10153.0000    10092.0000 50804621    0.60%
 293509 57635    10102.0000    75    10

 363355 72583    10092.0000   192    10153.0000    10092.0000 62954606    0.60%
 363984 72643    10104.0000    88    10153.0000    10092.0000 63067681    0.60%
 364753 72639    10092.0000    91    10153.0000    10092.0000 62975933    0.60%
 365364 72646    infeasible          10153.0000    10092.0000 63101281    0.60%
Elapsed time = 13144.80 sec. (7052814.44 ticks, tree = 696.03 MB, solutions = 6)
 365873 72879    infeasible          10153.0000    10092.0000 63323461    0.60%
 366422 72892    infeasible          10153.0000    10092.0000 63270514    0.60%
 366702 73335        cutoff          10153.0000    10092.0000 64111270    0.60%
 366846 73318    10092.0000    99    10153.0000    10092.0000 64095939    0.60%
 366974 73295    infeasible          10153.0000    10092.0000 64013405    0.60%
 367150 73439    infeasible          10153.0000    10092.0000 64277389    0.60%
 367271 73422    10092.0000   100    10153.0000    10092.0000 64328596    0.60%
 367419 73423    10092.0000    95    10

Elapsed time = 15959.66 sec. (8434967.21 ticks, tree = 739.75 MB, solutions = 6)
 399027 77789    10092.0395    65    10153.0000    10092.0000 74521646    0.60%
 399123 77791    infeasible          10153.0000    10092.0000 74526038    0.60%
 399210 77790    10092.0000    71    10153.0000    10092.0000 74597525    0.60%
 399330 77802    10092.0000   117    10153.0000    10092.0000 74878630    0.60%
 399592 77823    10092.0000   136    10153.0000    10092.0000 74914413    0.60%
 399824 77889    10099.0000    48    10153.0000    10092.0000 75041316    0.60%
 400276 77882    10092.0000   145    10153.0000    10092.0000 75032278    0.60%
 400936 77905    infeasible          10153.0000    10092.0000 75070865    0.60%
 401600 77884    infeasible          10153.0000    10092.0000 75073348    0.60%
 402070 78192        cutoff          10153.0000    10092.0000 75563513    0.60%
Elapsed time = 16325.73 sec. (8588900.64 ticks, tree = 742.75 MB, solutions = 6)
 402548 78140    10092.0000    79    1

 419847 80512    10103.0000    69    10153.0000    10092.0000 85058710    0.60%
 420384 80539    10115.0000    47    10153.0000    10092.0000 84892583    0.60%
 420953 80550        cutoff          10153.0000    10092.0000 84904882    0.60%
 421613 80714    infeasible          10153.0000    10092.0000 85353833    0.60%
 422105 80545    10092.0000   103    10153.0000    10092.0000 85120267    0.60%
 422472 80858    10092.0000    58    10153.0000    10092.0000 85685276    0.60%
 422869 80898    10092.0000   103    10153.0000    10092.0000 85763027    0.60%
Elapsed time = 19537.88 sec. (9971879.80 ticks, tree = 766.24 MB, solutions = 6)
 423245 80899    10092.0000    54    10153.0000    10092.0000 85725628    0.60%
 423418 80900    10092.0000    42    10153.0000    10092.0000 85731571    0.60%
 423675 81182    10092.0000    71    10153.0000    10092.0000 86297053    0.60%
 423753 80985    10092.0000   130    10153.0000    10092.0000 86045499    0.60%
 423848 81230    10093.0000    89    10

In [52]:
import json 
outDir = "tests/model/solutions/"
testInstanceDir = "tests/model/instances/"
for instanceId in np.arange(1,8):
    with open(outDir + "instance-" + str(instanceId) + ".json", "r") as solutionFile:
        jsonContent = solutionFile.readline()
        loadedSolution = json.loads(jsonContent)
        instanceFile = testInstanceDir + "instance-" + str(instanceId) + ".test"
        mdl = Model(name="3DBPP")
        problem = parseInstance(instanceFile)
        W = problem[0]
        D = problem[1]
        H = problem[2]
        V = W*D*H
        DU = 10
        bins = 1
        originalBoxes = problem[3]
        boxes = generateRotatedDuplices(originalBoxes)
        I_O = np.arange(len(originalBoxes))
        I = np.arange(len(boxes))
        B = np.arange(bins)
        alpha_s = 0.7
        beta_s = 5

        maxDim = max(max(boxes[i].w, boxes[i].d) for i in I)
        I_R = np.arange(-int(np.floor(maxDim/DU)), int(np.floor(maxDim/DU)))

        # Problem Variables
        v = [mdl.integer_var(0, 1, "v_" + str(b)) for b in B]
        u = [[mdl.integer_var(0, 1, "u_" + str(i) + "_" + str(b)) for b in B] for i in I]
        x = [mdl.continuous_var(0, W-min(boxes[i].w, boxes[i].d), "x_" + str(i)) for i in I]
        y = [mdl.continuous_var(0, D-min(boxes[i].w, boxes[i].d), "y_" + str(i)) for i in I]
        z = [mdl.continuous_var(0, H-boxes[i].h, "z_" + str(i)) for i in I]
        xp = [[mdl.integer_var(0, 1, "xp_" + str(i) + "_" + str(k)) for k in I] for i in I]
        yp = [[mdl.integer_var(0, 1, "yp_" + str(i) + "_" + str(k)) for k in I] for i in I]
        zp = [[mdl.integer_var(0, 1, "zp_" + str(i) + "_" + str(k)) for k in I] for i in I]
        zc = [[mdl.integer_var(0, 1, "zc_" + str(i) + "_" + str(k)) for k in I] for i in I]
        zmax = [mdl.continuous_var(0, H, "zmax_" + str(b)) for b in B]
        s = [[mdl.integer_var(0, 1, "s_" + str(i) + "_" + str(k)) for k in I] for i in I]
        g = [mdl.integer_var(0, 1, "g_" + str(i)) for i in I]
        sp = [[[[[mdl.integer_var(0, 1, "sp_" + str(i) + "_" + str(k)  + "_" + str(b) + "_" + str(dx) + "_" + str(dy)) for dy in I_R] for dx in I_R] for b in B ] for k in I] for i in I]

        #Objective
        mdl.minimize(mdl.sum((H*v[b] + zmax[b]) for b in B) - mdl.sum(sp[i][j][b][dx][dy] for dx in I_R for dy in I_R for b in B for i in I for j in I if calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0))                                                                                     #1

        #Constraints
        #[mdl.add_constraint(mdl.sum(u[i][b] for b in B) == 1) for i in I]
        [mdl.add_constraint(mdl.sum(u[i*2][b] for b in B) + mdl.sum(u[(i*2)+1][b] for b in B) == 1) for i in I_O]
        [mdl.add_constraint(u[i][b] <= v[b]) for b in B for i in I]

        [mdl.add_constraint(x[i] + boxes[i].w <= W) for i in I]
        [mdl.add_constraint(y[i] + boxes[i].d <= D) for i in I]
        [mdl.add_constraint(z[i] + boxes[i].h <= H) for i in I]

        [mdl.add_constraint(zmax[b] >= (z[i] + boxes[i].h) - H*(1-u[i][b])) for i in I for b in B]                                                             #9

        [mdl.add_constraint((x[i] + boxes[i].w) - x[j] <= W*(1-xp[i][j])) for i in I for j in I]
        [mdl.add_constraint(x[j] - (x[i] + boxes[i].w) + 1 <= W*xp[i][j]) for i in I for j in I]

        [mdl.add_constraint((y[i] + boxes[i].d) - y[j] <= D*(1-yp[i][j])) for i in I for j in I]
        [mdl.add_constraint(y[j] - (y[i] + boxes[i].d) + 1 <= D*yp[i][j]) for i in I for j in I]

        [mdl.add_constraint((z[i] + boxes[i].h) - z[j] <= H*(1-zp[i][j])) for i in I for j in I]
        [mdl.add_constraint(z[j] - (z[i] + boxes[i].h) + 1 <= H*zp[i][j]) for i in I for j in I]

        [mdl.add_constraint(v[b] >= v[c]) for b in B for c in B if c > b]

        [mdl.add_constraint(xp[i][j] + xp[j][i] +
                           yp[i][j] + yp[j][i] +
                           zp[i][j] + zp[j][i] >= u[i][b] + u[j][b] - 1) for i in I for j in I for b in B if not i == j]

        # Support constraint

        [mdl.add_constraint(z[j] - (z[i] + boxes[i].h) <= beta_s + H*(1-zc[i][j])) for i in I for j in I if i != j]
        [mdl.add_constraint(z[j] - (z[i] + boxes[i].h) >= -beta_s - H*(1-zc[i][j])) for i in I for j in I if i != j]
        [mdl.add_constraint(s[i][j] <= zp[i][j]) for i in I for j in I]
        [mdl.add_constraint(s[i][j] <= zc[i][j]) for i in I for j in I]
        [mdl.add_constraint(s[i][j] >= zp[i][j] + zc[i][j] - 2) for i in I for j in I]
        [mdl.add_constraint(mdl.sum(s[i][j] for j in I) <= mdl.sum(u[i][b] for b in B)) for i in I]
        [mdl.add_constraint(z[i] <= beta_s + H*(1 - g[i])) for i in I]

        [mdl.add_constraint(mdl.sum(sp[i][j][b][dx][dy] for dx in I_R for dy in I_R for b in B if calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0) <= s[i][j]) for i in I for j in I]

        [mdl.add_constraint(x[i] - x[j] >= DU*dx - 2*W*(1 - sp[i][j][b][dx][dy])) for dx in I_R for dy in I_R for b in B for i in I for j in I if i != j and calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0]
        [mdl.add_constraint(x[i] - x[j] <= DU*(dx + 1) + 2*W*(1 - sp[i][j][b][dx][dy])) for dx in I_R for dy in I_R for b in B for i in I for j in I if i != j and calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0]
        [mdl.add_constraint(y[i] - y[j] >= DU*dy - 2*D*(1 - sp[i][j][b][dx][dy])) for dx in I_R for dy in I_R for b in B for i in I for j in I if i != j and calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0]
        [mdl.add_constraint(y[i] - y[j] <= DU*(dy + 1) + 2*D*(1 - sp[i][j][b][dx][dy])) for dx in I_R for dy in I_R for b in B for i in I for j in I if i != j and calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0]

        [mdl.add_constraint(mdl.sum(calculateOverlap(j, b, i, b, dx*DU, dy*DU)*sp[j][i][b][dx][dy] for dx in I_R for dy in I_R for b in B for j in I if i != j and calculateOverlap(j, b, i, b, dx*DU, dy*DU) != 0) >= alpha_s*boxes[i].w*boxes[i].d - boxes[i].w*boxes[i].d*g[i]) for i in I]

        class MyEncoder(json.JSONEncoder):
            def default(self, obj):
                if isinstance(obj, np.integer):
                    return int(obj)
                elif isinstance(obj, np.floating):
                    return float(obj)
                elif isinstance(obj, np.ndarray):
                    return obj.tolist()
                else:
                    return super(MyEncoder, self).default(obj)

        solution = mdl.new_solution()
        for var in loadedSolution["CPLEXSolution"]["variables"]:
            solution.add_var_value(var["name"], float(var["value"]))
        solutions = [getSolutionBox(i, boxes, solution) for i in I]
        nonNulls = [box for box in solutions if box != None]
        with open(outDir + "processed/instance-" + str(instanceId) + ".json", "w") as outFile:
            outFile.write(json.dumps([box.to_dict() for box in nonNulls], cls=MyEncoder))
        print(checkSolution(boxes, solution))

(True, '')
(True, '')
(True, '')
(True, '')
(True, '')
(True, '')
(False, 'Box 3 got area support 0.6584967320261436')
